In [1]:
#export
"""Small module with helper functions to analyze and manipulate Python abstract syntax tree"""
import k1lib, math, numpy as np, random, base64, json, time, ast; from typing import List, Iterator
from collections import defaultdict, deque; import k1lib.cli as cli
__all__ = ["walk", "pretty", "py2js", "asyncGuard"]

In [2]:
#export
class JSTranspileError(Exception): pass
def _walk(m, parent, accessor="", depth=0): # yields (elem, parent, parent accessor, depth). This function don't need to recurse leaf nodes, which is a pretty good time saver
    yield m, parent, accessor, depth
    if isinstance(m, (ast.Module, ast.ClassDef, ast.FunctionDef)):
        for expr in m.body: yield from _walk(expr, m, f".body[{len(m.body)}]", depth+1)
    elif isinstance(m, ast.Expr):
        yield from _walk(m.value, m, ".value", depth+1)
    elif isinstance(m, ast.Lambda):
        yield from _walk(m.body, m, ".body", depth+1)
    elif isinstance(m, ast.BinOp):
        yield from _walk(m.left, m, ".left", depth+1)
        yield from _walk(m.op, m, ".op", depth+1)
        yield from _walk(m.right, m, ".right", depth+1)
    elif isinstance(m, ast.Assign):
        yield from _walk(m.value, m, ".value", depth+1)
        for e in m.targets: yield from _walk(e, m, f".targets[{len(m.targets)}]", depth+1)
    elif isinstance(m, ast.If):
        yield from _walk(m.test, m, ".test", depth+1)
        for e in m.body: yield from _walk(e, m, f".body[{len(m.body)}]", depth+1)
        for e in m.orelse: yield from _walk(e, m, f".orelse[{len(m.orelse)}]", depth+1)
    elif isinstance(m, ast.Call):
        yield from _walk(m.func, m, ".func", depth+1)
        for e in m.args: yield from _walk(e, m, f".args[{len(m.args)}]", depth+1)
        for e in m.keywords: yield from _walk(e, m, f".keywords[{len(m.keywords)}]", depth+1)
    elif isinstance(m, ast.Attribute):
        yield from _walk(m.value, m, ".value", depth+1)
    elif isinstance(m, ast.UnaryOp):
        yield from _walk(m.op, m, ".op", depth+1)
        yield from _walk(m.operand, m, ".operand", depth+1)
    elif isinstance(m, ast.Subscript):
        yield from _walk(m.value, m, ".value", depth+1)
        yield from _walk(m.slice, m, ".slice", depth+1)
        yield from _walk(m.ctx, m, ".ctx", depth+1)
    # elif isinstance(m, ast.Store):
    #     yield from _walk(m.parent, m, ".parent", depth+1)
    elif isinstance(m, ast.Compare):
        yield from _walk(m.left, m, ".left", depth+1)
        for e in m.ops: yield from _walk(e, m, f".ops[{len(m.ops)}]", depth+1)
        for e in m.comparators: yield from _walk(e, m, f".comparators[{len(m.comparators)}]", depth+1)
    elif isinstance(m, ast.List):
        for e in m.elts: yield from _walk(e, m, f".elts[{len(m.elts)}]", depth+1)
    elif isinstance(m, ast.Tuple):
        for e in m.elts: yield from _walk(e, m, f".elts[{len(m.elts)}]", depth+1)
    elif isinstance(m, ast.Set):
        for e in m.elts: yield from _walk(e, m, f".elts[{len(m.elts)}]", depth+1)
    elif isinstance(m, ast.comprehension):
        yield from _walk(m.is_async, m, ".is_async", depth+1)
        yield from _walk(m.iter, m, ".iter", depth+1)
        yield from _walk(m.target, m, ".target", depth+1)
        for e in m.ifs: yield from _walk(e, m, f".ifs[{len(m.ifs)}]", depth+1)
    elif isinstance(m, (ast.ListComp, ast.GeneratorExp, ast.SetComp)):
        yield from _walk(m.elt, m, ".elt", depth+1)
        for e in m.generators: yield from _walk(e, m, f".generators[{len(m.generators)}]", depth+1)
    elif isinstance(m, ast.DictComp):
        yield from _walk(m.key, m, ".key", depth+1)
        yield from _walk(m.value, m, ".value", depth+1)
        for e in m.generators: yield from _walk(e, m, f".generators[{len(m.generators)}]", depth+1)
    elif isinstance(m, ast.Starred):
        yield from _walk(m.ctx, m, ".ctx", depth+1)
        yield from _walk(m.value, m, ".value", depth+1)
    elif isinstance(m, ast.keyword):
        yield from _walk(m.value, m, ".value", depth+1)
    elif isinstance(m, ast.Dict):
        for e in m.keys: yield from _walk(e, m, f".keys[{len(m.keys)}]", depth+1)
        for e in m.values: yield from _walk(e, m, f".values[{len(m.values)}]", depth+1)
    elif isinstance(m, ast.IfExp):
        yield from _walk(m.body, m, ".body", depth+1)
        yield from _walk(m.test, m, ".test", depth+1)
        yield from _walk(m.orelse, m, ".orelse", depth+1)
    elif isinstance(m, ast.Slice):
        yield from _walk(m.lower, m, ".lower", depth+1)
        yield from _walk(m.upper, m, ".upper", depth+1)
        yield from _walk(m.step, m, ".step", depth+1)
    elif isinstance(m, ast.FormattedValue):
        yield from _walk(m.value, m, ".value", depth+1)
    elif isinstance(m, ast.JoinedStr):
        for e in m.values: yield from _walk(e, m, f".values[{len(m.values)}]", depth+1)
class walk(cli.BaseCli):
    def __init__(self):
        """Walks the AST tree, returning (elem, parent, parent accessor, depth) tuple.
Example::

    "abc.py" | cat() | join("\\n") | kast.walk() # returns generator of tuples
"""
        pass
    def __ror__(self, m:str):
        if isinstance(m, str): m = ast.parse(m)
        return _walk(m, None, "", 0)

In [3]:
#export
class pretty(cli.BaseCli):
    def __init__(self):
        """Returns list[str] containing the prettified tree structure of the AST. Mainly
for asthetic, not for parsing by other tools downstream. Example::

    "x+3" | kast.pretty()
    "lambda x: x+3" | kast.pretty()
"""
        pass
    def __ror__(self, it:str):
        m = it; res = []
        if isinstance(m, str): m = ast.parse(m)
        for m, parent, accessor, depth in _walk(m, None, "", 0):
            pad = "  "*depth; res.append(f"{pad}{accessor} {m}")
            if isinstance(m, ast.arguments): res[-1] += f" {[a.arg for a in m.args]}"
            elif isinstance(m, ast.Import): res[-1] += f" .names[{len(m.names)}].name={', '.join([e.name for e in m.names])}"
            elif isinstance(m, ast.ImportFrom): res[-1] += f" .module={repr(m.module)} .names[{len(m.names)}].name={', '.join([e.name for e in m.names])}"
            elif isinstance(m, ast.BinOp): res[-1] += f" .op={m.op}"
            elif isinstance(m, ast.Lambda): res[-1] += f" .args={[a.arg for a in m.args.args]}"
            elif isinstance(m, (ast.ClassDef, ast.FunctionDef)): res[-1] += f" .name={repr(m.name)}"
            elif isinstance(m, ast.Name): res[-1] += f" .id={repr(m.id)}"
            elif isinstance(m, ast.Constant): res[-1] += f" .value={repr(m.value)}"
            elif isinstance(m, ast.Attribute): res[-1] += f" .attr={repr(m.attr)}"
            elif isinstance(m, ast.keyword): res[-1] += f" .arg={repr(m.arg)}"
        return res

In [4]:
#export
jstCallDict = {}
def _jstGuardNArgs(name, ns, f): # common guard across all short and simple functions
    ns = set(ns)
    if len(ns) == 0: jstCallDict[name] = f
    else:
        def inner(m, lcs):
            if len(m.args) not in ns: raise JSTranspileError(f"Function `{name}` only support these number of arguments: {ns}. You have {len(m.args)} arguments total!")
            return f(m, lcs)
        jstCallDict[name] = inner
_jstGuardNArgs("abs", [1], lambda m, lcs: f"Math.abs({_py2js(m.args[0], lcs)})")
_jstGuardNArgs("all", [1], lambda m, lcs: f"{_py2js(m.args[0], lcs)}.every((x) => x)")
_jstGuardNArgs("any", [1], lambda m, lcs: f"{_py2js(m.args[0], lcs)}.some((x) => x)")
def _jst_divmod(m, lcs):
    if len(m.args) != 2: raise JSTranspileError(f"Only support divmod() with 2 arguments!")
    a = _py2js(m.args[0], lcs); b = _py2js(m.args[1], lcs)
    return f"[Math.floor(a/b), a-b*Math.floor(a/b)]"
jstCallDict["divmod"] = _jst_divmod
def _jst_enumerate(m, lcs):
    if len(m.args) != 1: raise JSTranspileError("Currently don't support enumerate() with the start parameter")
    return f"{_py2js(m.args[0], lcs)}.map((x, i) => [x, i])"
jstCallDict["enumerate"] = _jst_enumerate
_jstGuardNArgs("float", [1], lambda m, lcs: f"parseFloat({_py2js(m.args[0], lcs)})")
def _jst_getattr(m, lcs):
    a = _py2js(m.args[0], lcs); b = _py2js(m.args[1], lcs)
    if len(m.args) == 2: return f"{a}[{b}]"
    else: return f"{a}[{b}] ?? {_py2js(m.args[2], lcs)}"
jstCallDict["getattr"] = _jst_getattr
_jstGuardNArgs("hasattr", [2], lambda m, lcs: f"{_py2js(m.args[0], lcs)}[{_py2js(m.args[1], lcs)}] !== undefined")
_jstGuardNArgs("int", [1], lambda m, lcs: f"parseInt({_py2js(m.args[0], lcs)})")
_jstGuardNArgs("iter", [1], lambda m, lcs: f"Array.from({_py2js(m.args[0], lcs)})")
_jstGuardNArgs("len", [1], lambda m, lcs: f"{_py2js(m.args[0], lcs)}.length")
_jstGuardNArgs("list", [1], lambda m, lcs: f"Array.from({_py2js(m.args[0], lcs)})")
def _jst_max(m, lcs):
    if len(m.args) == 0: raise JSTranspileError("Only support max() with 1 or more parameters")
    if len(m.args) == 1: return f"Math.max(...({_py2js(m.args[0], lcs)}))"
    return f"Math.max({', '.join([_py2js(e, lcs) for e in m.args])})"
jstCallDict["max"] = _jst_max
def _jst_min(m, lcs):
    if len(m.args) == 0: raise JSTranspileError("Only support min() with 1 or more parameters")
    if len(m.args) == 1: return f"Math.min(...({_py2js(m.args[0], lcs)}))"
    return f"Math.min({', '.join([_py2js(e, lcs) for e in m.args])})"
jstCallDict["min"] = _jst_min
_jstGuardNArgs("pow", [2], lambda m, lcs: f"Math.pow({_py2js(m.args[0], lcs)}, {_py2js(m.args[1], lcs)})")
_jstGuardNArgs("print", [], lambda m, lcs: f"console.log({', '.join([_py2js(e, lcs) for e in m.args])})")
def _jst_range(m, lcs):
    n = len(m.args)
    if n == 1: start = None; stop = m.args[0]
    elif n == 2: start, stop = m.args
    else: raise JSTranspileError("Only support range() without the step variable")
    ans = f"[...Array({_py2js(stop, lcs)}).keys()]"
    if start is not None: ans = f"{ans}.slice({_py2js(start, lcs)})"
    return ans
jstCallDict["range"] = _jst_range
_jstGuardNArgs("reversed", [1], lambda m, lcs: f"({_py2js(m.args[0], lcs)}).toReversed()")
def _jst_round(m, lcs):
    if len(m.args) > 2 or len(m.args) <= 0: raise JSTranspileError("Only support round() with 1 or 2 arguments!")
    if len(m.args) == 1: return f"Math.round({_py2js(m.args[0], lcs)})"
    else: b = _py2js(m.args[1], lcs); return f"Math.round(({_py2js(m.args[0], lcs)})*Math.pow(10, {b})+Number.EPSILON)/Math.pow(10, {b})"
jstCallDict["round"] = _jst_round
_jstGuardNArgs("set", [1], lambda m, lcs: f"new Set({_py2js(m.args[0], lcs)})")
_jstGuardNArgs("setattr", [3], lambda m, lcs: f"({_py2js(m.args[0], lcs)}[{_py2js(m.args[1], lcs)}] = {_py2js(m.args[2], lcs)})")
# sorted, ignored cause user can just use sort() cli instead! Python's sort() has weird args
_jstGuardNArgs("str", [1], lambda m, lcs: f"({_py2js(m.args[0], lcs)}).toString()")
_jstGuardNArgs("sum", [1], lambda m, lcs: f"({_py2js(m.args[0], lcs)}).reduce((partialSum, a) => partialSum + a, 0)")
_jstGuardNArgs("tuple", [1], lambda m, lcs: f"Array.from({_py2js(m.args[0], lcs)})")
_jstGuardNArgs("type", [1], lambda m, lcs: f"typeof({_py2js(m.args[0], lcs)})")
_jstGuardNArgs("zip", [], lambda m, lcs: f"[{', '.join([_py2js(e, lcs) for e in m.args])}].transpose()")

In [13]:
#export
def flattenAttribute(x) -> str:
    if isinstance(x, ast.Name): return x.id
    elif isinstance(x, ast.Attribute): return f"{flattenAttribute(x.value)}.{x.attr}"
    else: raise JSTranspileError(f"kast.flattenAttribute() only supports attributes that only contains ast.Attribute and ast.Name. Instead input is of type {type(x)}")
def pyLambParse(s:str) -> "ast.AST": # parses code, get rid of "lambda: " in front if exists
    # s might contain stringified 
    m = ast.parse(s)
    if len(m.body) == 0: raise JSTranspileError(f"No expression inside passed in code `{s}`")
    if len(m.body) > 1: raise JSTranspileError(f"py2js() function can only work with relatively simple operations. The code passed in is too complicated: `{s}`")
    expr = m.body[0].value
    if isinstance(expr, ast.Lambda): expr = expr.body
    return expr
def py2js(m):
    """Converts simple python expression into js code.
Example::

    kast.py2js("x+3")                 # returns ('(x+3)', {'x'})
    kast.py2js("lambda x: x+3")       # returns ('(x+3)', {'x'})
    kast.py2js("lambda y: y+3")       # returns ('(y+3)', {'y'})
    kast.py2js("lambda y: y+3*x")     # returns ('(y+(3*x))', {'x', 'y'})
    kast.py2js("x<3 and 45>10")       # returns ('((x<3)&&(45>10))', {'x'})
    kast.py2js("abs((x//5.1) * 2.3)") # returns ('Math.abs((Math.floor(x/5.1)*2.3))', {'x'})
    kast.py2js("not x > 3")           # returns ('!(x>3)', {'x'})

This will return a tuple of 2 objects. First is the JS code, and second is the
set of unknown variables in the JS code. If the code is a lambda function,
then it only considers what's inside the function. More complex examples::

    # returns ('[...Array(10).keys()].map((x, i) => [x, i]).filter(([x, y]) => ((x%3)===1)).map(([x, y]) => (Math.pow(x, y)+z))', {'z'})
    kast.py2js("[x**y + z for x, y in enumerate(range(10)) if x % 3 == 1]")

Both JS and Python code when run should return [1, 256, 823543] for z = 0.

A lot of builtin Python functions are transpiled automatically, including::

    abs, all, any, divmod, enumerate, float, getattr, hasattr, int, iter
    len, list, max, min, pow, print, range, reversed, round, set, setattr
    str, sum, tuple, type, zip

Other builtin operations don't quite make sense though. But the neat thing
is, any functions beyond these builtin functions are not transpiled, and
instead just passes along to the JS code. This allows you to kinda mix and
match code styles from both Python and JS. For example::

    # returns ("(document.querySelector(('#' + someIdx + '_tail')).style['color'] = (((Math.round(Math.pow(x, 2))===Math.abs(-12))) ? ('red') : ('green')))", {'someIdx', 'x'})
    kast.py2js("setattr(document.querySelector(f'#{someIdx}_tail').style, 'color', 'red' if round(x**2) == Math.abs(-12) else 'green')")

See how we're both using the Python transpiled ``**`` (to ``Math.pow(...)``),
yet uses ``Math.abs(...)`` directly, which is not defined in Python, but is
defined in JS, without any trouble. Also notice that ``document.querySelector``
and f-strings work too. So in short, overall syntax is Python's, but you can
call random JS functions!

This can do lots of transformations, but don't expect it to be airtight. I've
done all I can to make the thing airtight, but I'm sure you can come up with
some complex scenario where it would fail. This is meant to be applied on
short pieces of code only."""
    lcs = set(); ans = _py2js(m, lcs); return ans, lcs
def _listCompTarget(m, lcs): # extracts target of list comprehensions, like [... for x, [y, z] in ...]
    if isinstance(m, ast.Name): return _py2js(m, lcs)
    if isinstance(m, (ast.Tuple, ast.List)): s = ", ".join([_listCompTarget(e, lcs) for e in m.elts]); return f"[{s}]"
    raise JSTranspileError(f"While trying to expand target assignment in list/set/dict comprehensions, encountered unknown type: {type(m)}")
def _py2js(m:"ast.AST|str", lcs:"list[str]") -> str: # lcs for list of variable names, only used in ast.Name
    if isinstance(m, str): m = pyLambParse(m)
    if isinstance(m, ast.BinOp):
        left = _py2js(m.left, lcs); right = _py2js(m.right, lcs)
        if isinstance(m.op, ast.FloorDiv): return f"Math.floor({left}/{right})" # TODO: might be weird dealing with negative floors
        if isinstance(m.op, ast.Pow): return f"Math.pow({left}, {right})"
        return f"({left}{_py2js(m.op, lcs)}{right})"
    if isinstance(m, ast.Compare): # only supports a<b or a<b<c, dont support more elems, like a<b<c<d
        nops = len(m.ops); ncomps = len(m.comparators)
        if nops != ncomps: raise JSTranspileError(f"Number of operations {nops} differed from number of comparators {ncomps}")
        if nops == 0: raise JSTranspileError("Number of operations can't be zero")
        if nops > 2: raise JSTranspileError("Don't support Compares with more than 2 comparisons. Change `a<b<c<d` into `(a<b<c) and (c<d)`")
        left = _py2js(m.left, lcs); comps = [_py2js(e, lcs) for e in m.comparators]; ops = [_py2js(o, lcs) for o in m.ops]
        if nops == 1: return f"({left}{ops[0]}{comps[0]})"
        if nops == 2: return f"({left}{ops[0]}{comps[0]}) && ({comps[0]}{ops[1]}{comps[1]})"
        return m
    if isinstance(m, ast.BoolOp): return f"({_py2js(m.op, lcs).join([_py2js(v, lcs) for v in m.values])})"
    if isinstance(m, ast.UnaryOp):
        if isinstance(m.op, ast.Not): return f"!{_py2js(m.operand, lcs)}"
        if isinstance(m.op, ast.Invert): raise JSTranspileError(f"No notion of inversion '~' exists in JS")
        if isinstance(m.op, ast.USub): return f"-{_py2js(m.operand, lcs)}"
        if isinstance(m.op, ast.UAdd): return f"{_py2js(m.operand, lcs)}"
        raise JSTranspileError(f"kast.py2js() doesn't know how to parse UnaryOp `{m.op}`")
    if isinstance(m, ast.Name): lcs.add(m.id); return f"{m.id}"
    if isinstance(m, ast.Constant):
        if m.value is None: return f"null"
        if isinstance(m.value, str): s = m.value.replace("'", "\\'"); return f"'{s}'"
        return f"{m.value}"
    if isinstance(m, ast.Add): return f"+"
    if isinstance(m, ast.Sub): return f"-"
    if isinstance(m, ast.Mult): return f"*"
    if isinstance(m, ast.MatMult): raise JSTranspileError("No notion of matrix multiplication '@' exists in JS")
    if isinstance(m, ast.Div): return f"/"
    if isinstance(m, ast.Mod): return f"%"
    if isinstance(m, ast.Pow): return f"^"
    if isinstance(m, ast.Eq): return f"==="
    if isinstance(m, ast.NotEq): return f"!=="
    if isinstance(m, ast.Gt): return f">"
    if isinstance(m, ast.GtE): return f">="
    if isinstance(m, ast.Lt): return f"<"
    if isinstance(m, ast.LtE): return f"<="
    if isinstance(m, ast.LShift): return "<<"
    if isinstance(m, ast.RShift): return ">>"
    if isinstance(m, ast.And): return "&&"
    if isinstance(m, ast.Or): return "||"
    if isinstance(m, ast.Attribute): return f"{_py2js(m.value, lcs)}.{m.attr}"
    if isinstance(m, ast.Call):
        if len(m.keywords) > 0: raise JSTranspileError(f"keyword arguments don't exist in JS, please use normal arguments in your function calls only, at `{m}`. Keywords: {[k.arg for k in m.keywords]}")
        funcName = flattenAttribute(m.func)
        if funcName in jstCallDict: return jstCallDict[funcName](m, lcs)
        return f"{funcName}({', '.join([_py2js(a, lcs) for a in m.args])})"
    if isinstance(m, (ast.List, ast.Tuple)): return f"[{', '.join([_py2js(e, lcs) for e in m.elts])}]"
    if isinstance(m, ast.Set): return f"new Set([{', '.join([_py2js(e, lcs) for e in m.elts])}])"
    if isinstance(m, ast.Dict):
        keys = [f"{_py2js(k, lcs)}" if isinstance(k, ast.Constant) else (f"[{_py2js(k, lcs)}]" if k is not None else None) for k in m.keys]
        values = [f"{_py2js(v, lcs)}" for v in m.values]
        a = [(f"...{v}" if k is None else f"{k}: {v}") for k, v in zip(keys, values)]; return "{" + ", ".join(a) + "}"
    if isinstance(m, ast.Starred): return f"...{_py2js(m.value, lcs)}"
    if isinstance(m, ast.IfExp): return f"(({_py2js(m.test, lcs)}) ? ({_py2js(m.body, lcs)}) : ({_py2js(m.orelse, lcs)}))"
    if isinstance(m, ast.Subscript):
        tail = ""; slices = m.slice.elts if isinstance(m.slice, ast.Tuple) else [m.slice]
        for s in reversed(slices):
            if isinstance(s, ast.Slice):
                if s.step is not None: pr = '\n'.join(m | pretty()); raise JSTranspileError(f"JS does not support stepped slices! This can be augmented on my side later on though. kast.pretty() on the element:\n\n{pr}")
                l = 'undefined' if s.lower is None else _py2js(s.lower, lcs)
                u = 'undefined' if s.upper is None else _py2js(s.upper, lcs)
                if tail: dataIdx = k1lib.cli.init._jsDAuto(); tail = f".slice({l}, {u}).map(({dataIdx}) => {dataIdx}{tail})"
                else: tail = f".slice({l}, {u})" # short path
            else: tail = f".at({_py2js(s, lcs)}){tail}"
        return f"{_py2js(m.value, lcs)}{tail}"
    if isinstance(m, (ast.ListComp, ast.GeneratorExp, ast.SetComp)):
        if len(m.generators) != 1: raise JSTranspileError("Currently only supports 1 generator in list/set/generator comprehensions")
        g = m.generators[0]; targetLcs = set(); oldLcs = set(lcs)
        if g.is_async: raise JSTranspileError("Does not support async functions yet")
        ans = _py2js(g.iter, lcs); target = _listCompTarget(g.target, targetLcs)
        for if_ in g.ifs: ans = f"{ans}.filter(({target}) => {_py2js(if_, lcs)})"
        ans = f"{ans}.map(({target}) => {_py2js(m.elt, lcs)})"
        for x in list(lcs): # try removing unknown variables in the .elt expression that appears in the target, cause those are not unknown variables!
            if x not in oldLcs and x in targetLcs: lcs.remove(x)
        return f"new Set({ans})" if isinstance(m, ast.SetComp) else ans
    if isinstance(m, ast.DictComp):
        if len(m.generators) != 1: raise JSTranspileError("Currently only supports 1 generator in dict comprehensions")
        g = m.generators[0]; targetLcs = set(); oldLcs = set(lcs)
        if g.is_async: raise JSTranspileError("Does not support async functions yet")
        ans = _py2js(g.iter, lcs); target = _listCompTarget(g.target, targetLcs)
        for if_ in g.ifs: ans = f"{ans}.filter(({target}) => {_py2js(if_, lcs)})"
        ans = f"{ans}.map(({target}) => [{_py2js(m.key, lcs)}, {_py2js(m.value, lcs)}]).toDict()"
        for x in list(lcs): # try removing unknown variables in the .elt expression that appears in the target, cause those are not unknown variables!
            if x not in oldLcs and x in targetLcs: lcs.remove(x)
        return ans
    if isinstance(m, ast.FormattedValue): return _py2js(m.value, lcs)
    if isinstance(m, ast.JoinedStr): return f"({' + '.join([_py2js(e, lcs) for e in m.values])})"
    raise JSTranspileError(f"kast.py2js() doesn't know how to parse {type(m)}")

The way to think about these tests is that they're not meant as a requirement, like "these absolutely must pass". Rather, it's to detect changes, and if the assertions fail, then you should look into it, verify that the output looks relatively reasonable, then rewrite the tests.

In [6]:
"setattr(document.querySelector('#abc').style, 'color', 'red' if x else 'green')" | pretty()

[' <ast.Module object at 0x7f8609efde20>',
 '  .body[1] <ast.Expr object at 0x7f8609efddf0>',
 '    .value <ast.Call object at 0x7f8609efdd60>',
 "      .func <ast.Name object at 0x7f8609efd100> .id='setattr'",
 "      .args[3] <ast.Attribute object at 0x7f8609efd0d0> .attr='style'",
 '        .value <ast.Call object at 0x7f8609efdfd0>',
 "          .func <ast.Attribute object at 0x7f8609efde50> .attr='querySelector'",
 "            .value <ast.Name object at 0x7f8609efde80> .id='document'",
 "          .args[1] <ast.Constant object at 0x7f8609efdeb0> .value='#abc'",
 "      .args[3] <ast.Constant object at 0x7f8609efdee0> .value='color'",
 '      .args[3] <ast.IfExp object at 0x7f8609efdd90>',
 "        .body <ast.Constant object at 0x7f8609efdc10> .value='red'",
 "        .test <ast.Name object at 0x7f8609efd310> .id='x'",
 "        .orelse <ast.Constant object at 0x7f8609efdbb0> .value='green'"]

In [7]:
assert py2js("x+3") == ('(x+3)', {'x'})
assert py2js("x<3 and 45>10") == ('((x<3)&&(45>10))', {'x'})
assert py2js("abs((x//5.1) * 2.3)") == ('Math.abs((Math.floor(x/5.1)*2.3))', {'x'})
assert py2js("not x > 3") == ('!(x>3)', {'x'})
assert py2js("[x, 3, 4, y]") == ('[x, 3, 4, y]', {'x', 'y'})
assert py2js("[x, 3, *z, 4, y]") == ('[x, 3, ...z, 4, y]', {'x', 'y', 'z'})
assert py2js("{'x': 3, 'y': 4}") == ("{'x': 3, 'y': 4}", set())
assert py2js("{'x': 3, y: 4}") == ("{'x': 3, [y]: 4}", {'y'})
assert py2js("{'z': 3, y-2: x**2}") == ("{'z': 3, [(y-2)]: Math.pow(x, 2)}", {'x', 'y'})
assert py2js("{'x': 3, y: 4, **z}") == ("{'x': 3, [y]: 4, ...z}", {'y', 'z'})
assert py2js("x if y else 4") == ('((y) ? (x) : (4))', {'x', 'y'})
assert py2js("x[3]") == ('x.at(3)', {'x'})
assert py2js("[*x, 3][y**2]") == ('[...x, 3].at(Math.pow(y, 2))', {'x', 'y'})
assert py2js("x[:3]") == ('x.slice(undefined, 3)', {'x'})
try: py2js("x[3:4:2]"); raise Exception("Failed")
except JSTranspileError: pass
py2js("x[:3, 6, 5:]")
assert py2js("range(x, y)") == ('[...Array(y).keys()].slice(x)', {'x', 'y'})
assert py2js("range(x)") == ('[...Array(x).keys()]', {'x'})
assert py2js("[x**2 for y in range(2, 10)]") == py2js("(x**2 for y in range(2, 10))")
assert py2js("[x**2 for y in range(2, 10)]") == ('[...Array(10).keys()].slice(2).map((y) => Math.pow(x, 2))', {'x'})
assert py2js("{x**2 for y in range(2, 10)}") == ('new Set([...Array(10).keys()].slice(2).map((y) => Math.pow(x, 2)))', {'x'})
assert py2js("[x**y for (x, y), z in range(2, 10)]") == ('[...Array(10).keys()].slice(2).map(([[x, y], z]) => Math.pow(x, y))', set())
assert py2js("{x**2:z for x, y in range(2, 10)}") == ('[...Array(10).keys()].slice(2).map(([x, y]) => [Math.pow(x, 2), z]).toDict()', {'z'})
assert py2js("f'{3}'") == ('(3)', set())
assert py2js("f'{3}_'") == ("(3 + '_')", set())
# complicated, integrated examples:
assert py2js("[x**y for x in range(2, 30) if x % 2 == 1]") == ('[...Array(30).keys()].slice(2).filter((x) => ((x%2)===1)).map((x) => Math.pow(x, y))', {'y'})
assert py2js("{x[4]: y**2} if y else [3, 4, *x]")[0] == '((y) ? ({[x.at(4)]: Math.pow(y, 2)}) : ([3, 4, ...x]))'
assert py2js("f'abc {3}_|_{x**2} def'") == ("('abc ' + 3 + '_|_' + Math.pow(x, 2) + ' def')", {'x'})
assert py2js("[x**y + z for x, y in enumerate(range(10)) if x % 3 == 1]") == ('[...Array(10).keys()].map((x, i) => [x, i]).filter(([x, y]) => ((x%3)===1)).map(([x, y]) => (Math.pow(x, y)+z))', {'z'})
assert py2js("setattr(document.querySelector(f'#{someIdx}_tail').style, 'color', 'red' if round(x**2) == Math.abs(-12) else 'green')") == ("(document.querySelector(('#' + someIdx + '_tail')).style['color'] = (((Math.round(Math.pow(x, 2))===Math.abs(-12))) ? ('red') : ('green')))", {'someIdx', 'x'})

In [8]:
#export
def prepareFunc(fn:"str|cli.op", extraVars:"list[str]"=None, checks=True):
    """for toJsFunc() transpilation subsystem.
Returns (transpiled function, variable declarations, argument variables).
If can't understand the object passed in, return None

:param fn: assumes this is a cli.op(), a lambda function, or an expression inside a lambda function
:param extraVars: available variables that can be resolved from js
    function. Can come from arguments, or from js function declaration,
    or other places. Specify this to shut up the variable resolver checks"""
    g = cli.init._k1_global_frame(); avaiVals = dict(); reqVals = set(); extraVars = extraVars or []
    if isinstance(fn, cli.op):
        fn, vs = fn.ab_fastFS()
        for v, vv in vs.items(): reqVals.add(v); avaiVals[v] = vv
    elif not isinstance(fn, str): return None; raise Exception(f"Function inside apply() has to either be a cli.op(), or a string containg a Python expression. Instead, it's {type(fn)}")
    # adds args from lambda func to available vars
    argVars = kast_lambda(fn); extraVars.extend(argVars)
    # transpiles to js
    fn, vs = py2js(fn)
    # try to resolve variables
    for v in vs:
        reqVals.add(v)
        if v == "x": continue # just ignore "x" variable, cause that's the default
        try: avaiVals[v] = g[v]
        except: pass
    # removing available variables from required variables
    for v in avaiVals: reqVals.remove(v)
    extraVars = set(extraVars)
    if checks:
        for v in reqVals:
            if v not in extraVars:
                raise Exception(f"Expression depends on the variables {v}, but they couldn't be resolved")
    # forming declaration statements
    return fn, "\n".join([f"{k} = {json.dumps(v)};" for k, v in avaiVals.items()]), argVars
def kast_lambda(fn:"str|cli.op") -> "list[str]":
    """Grab args of the input lambda func. If it's just a simple
expression then return ['x']"""
    l = ast.parse(fn).body[0].value
    if isinstance(l, ast.Lambda): return [a.arg for a in l.args.args]
    return ["x"]

In [15]:
assert kast_lambda("lambda a,b: b**a") == ["a", "b"]
assert kast_lambda("b**a") == ["x"]
assert kast_lambda(cli.op()*3) == ["x"]

TypeError: compile() arg 1 must be a string, bytes or AST object

In [16]:
#export
def prepareFunc2(fn, checks=True) -> "(header, fIdx) | None":
    res = prepareFunc(fn, [], checks)
    if res is None: return None
    expr, avaiVars, args = res
    fIdx = k1lib.cli.init._jsFAuto()
    dataIdx = k1lib.cli.init._jsDAuto()
    return f"""{avaiVars}
{fIdx} = ({', '.join(args)}) => {{
    return {expr};
}}""", fIdx
def prepareFunc3(fn, meta, kwargs=None, args=None) -> "(header, fIdx)":
    """Kinda similar to prepareFunc2(), but this time uses 3 different
strategies to transpile:

- Using prepareFunc2(), works for string with lambda functions and cli.op()
- Using the object's builtin ._jsF() function
- Using k1lib.settings.cli.kjs.jsF dictionary of different custom _jsF() functions

Throws error if can't transpile the function

:param kwargs: keyword arguments and arguments provided by aS(), apply(), filt(), and so on.
"""
    res = prepareFunc2(fn, False) # tries to compile str/cli.op
    if res:
        if not(kwargs is None or len(kwargs) == 0): raise Exception(f"Keyword arguments doesn't exist in JS! Please convert the function `{fn}` into positional arguments. Here're the kwargs detected: {kwargs}")
        if args is None or len(args) == 0: return res
        # prepare extra code here if extra args are available
        fIdx = cli.init._jsFAuto(); dataIdx = cli.init._jsDAuto(); argIdx = cli.init._jsDAuto()
        _header, _fIdx = res; return f"""{_header}\n{argIdx} = {json.dumps(args)};\n{fIdx} = ({dataIdx}) => {_fIdx}({dataIdx}, ...{argIdx})""", fIdx
    elif hasattr(fn, "_jsF"): return fn._jsF(meta, *(args or ()), **(kwargs or {})) # if not str/cli.op, then lookup the funcs, then pass through the kwargs and whatnot
    elif fn in k1lib.settings.cli.kjs.jsF:
        try: return k1lib.settings.cli.kjs.jsF[fn](meta, *(args or ()), **(kwargs or {}))
        except Exception as e: print(f"Function `{fn}` exist in system, but throws the below error when executed"); raise e
    else: raise Exception(f"*._jsF() right now doesn't know how to transpile to JS this func: {fn}\n\nIt could be you're using regular lambda functions. Convert it to use cli.op(), or just enter the lambda function as a string, so `lambda x: x**2` turns into the string `\"lambda x: x**2\"`\n\nOr, if it's a function you use often, add to `settings.cli.kjs.jsF`")

In [17]:
#export
def asyncGuard(data:"(header, fIdx)") -> "(header, fIdx, async?)":
    """Converts ._jsF() function signatures from (header, fIdx) into (header, fIdx, is fIdx async?).
Example::

    # returns tuple (header, fIdx, async?)
    kast.asyncGuard(apply("x**2")._jsF(None))
"""
    if data is NotImplemented: return NotImplemented
    header, fIdx = data
    g1 = cli.grep(f"{fIdx}[ ]*=[ ]*(?P<g>(async)|((?!\\(*)))[ ]*\\(", extract="g")
    g2 = cli.grep(f"(?P<g>async)[ ]*function[ ]*{fIdx}\\(", extract="g")
    x = header.split("\n"); return [header, fIdx, len(list(g1(x))) + len(list(g2(x))) > 0]

In [18]:
assert asyncGuard(["some\nother\nstuff\nhere\n_jsF_449_6 = (params) => 4", "_jsF_449_6"])[2] == 0
assert asyncGuard(["some\nother\nstuff\nhere\n_jsF_449_6 = async  (params) => 4", "_jsF_449_6"])[2] == 1
assert asyncGuard(["some\nother\nstuff\nhere\nasync function _jsF_449_6(param1) {\nsomething inside\n}", "_jsF_449_6"])[2] == 1
assert asyncGuard(["some\nother\nstuff\nhere\nfunction _jsF_449_6(param1) {\nsomething inside\n}", "_jsF_449_6"])[2] == 0

In [19]:
!../export.py kast --upload=True

2024-03-17 15:17:14,294	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-17 15:17:14,304	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15771   0   61%   
10172   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [14]:
!../export.py kast

2024-03-08 06:23:06,585	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-08 06:23:06,595	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15670   0   61%   
10026   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.6
Uninstalling k1lib-1.6:
  Successfully uninstalled k1lib-1.6
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [16]:
!../export.py kast --bootstrap=True

Traceback (most recent call last):
  File "/home/kelvin/repos/labs/k1lib/k1lib/../export.py", line 10, in <module>
    try: from k1lib.imports import *; hasK1 = True
  File "/home/kelvin/repos/labs/k1lib/k1lib/__init__.py", line 27, in <module>
    from . import kast;     kast     = wrapMod(kast)
  File "/home/kelvin/repos/labs/k1lib/k1lib/kast.py", line 404
    if not(kwargs is None or len(kwargs) == 0): raise Exception(f"Keyword arguments doesn't exist in JS! Please convert the function `{fn}` into positional arguments. Here're the kwargs detected: {}") # prepareFunc3
                                                                                                                                                                                                       ^
SyntaxError: f-string: empty expression not allowed
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/../